In [1]:
from ko_text import *

In [2]:
def load_data(path_to_file, usecols = ''):
    print('==================================================')
    print('Data Loading : %s'%path_to_file)
    try:
        if usecols:
            data = pd.read_csv(open(path_to_file,'r'), encoding='utf-8', engine='c', usecols = usecols)
        else:
            data = pd.read_csv(open(path_to_file,'r'), encoding='utf-8', engine='c')
        # Token 칼럼이 존재하는 경우
        if 'Token' in data.columns:
            # 용량을 줄이기 위해 '단어 단어' 꼴로 묶어둔 token을 ['단어', '단어']의 리스트 형태로 풀기
            data['Token'] = [token.split() for token in data['Token']]
        print('Data Loaded')

        return data

    except:
        print('%s가 존재하지 않습니다.'%path_to_file)


In [3]:
temp_df = load_data('../data/data_merged.csv', usecols = ['Section','Token'])

Data Loading : ../data/data_merged.csv
Data Loaded


In [4]:
nlp = NLP()
d2v = D2V('../Doc2Vec_model/Doc2Vec_dm=True&cc=6442&vs=100&win=5&min=1&sample=0.0001&epochs=10&dm_mean=0')

학습된 모델을 성공적으로 불러왔습니다.


In [5]:
token_ls = temp_df['Token'].tolist()

In [6]:
batch_size = 1000

In [31]:
def multiprocessing_queue_put(func, queue, **kwargs):
    queue.put(func(**kwargs))
    return

In [52]:
os.path.isfile('../Doc2Vec_model/Doc2Vec_dm=True&cc=6442&vs=100&win=5&min=1&sample=0.0001&epochs=10&dm_mean=0')

True

In [ ]:
def infer_vectors_multiprocessing(doc_ls):
    queue_ls = []
    procs = []
    result_ls = []
    batch_size = len(doc_ls) // 10
    
    # process에 작업들을 할당
    for i, idx in enumerate(range(0, len(token_ls), batch_size)):
        try:
            batch_link_map = token_ls[idx : idx + batch_size]
        except:
            batch_link_map = token_ls[idx :]

        queue_ls.append(Queue())
        proc = Process(
                target= multiprocessing_queue_put,
                kwargs = {
                    'func' : d2v.infer_vectors_with_Doc2Vec,
                    'queue' : queue_ls[i],
                    'doc_ls' : batch_link_map})

        procs.append(proc)    
        proc.start()

    for queue in queue_ls:
        temp_result_ls = queue.get()
        queue.close()
        del queue

        result_ls += temp_result_ls

    for proc in procs:
        proc.join()
        del proc

    return result_ls

In [49]:
%%time
result_ls = infer_vectors_multiprocessing(token_ls)

CPU times: user 51.2 ms, sys: 93.5 ms, total: 145 ms
Wall time: 18.3 s


In [39]:
%%time
result_ls = d2v.infer_vectors_with_Doc2Vec(token_ls)

CPU times: user 58.7 s, sys: 24.7 ms, total: 58.8 s
Wall time: 58.7 s


In [37]:
len(result_ls)

6442

In [ ]:
len(result_ls)

In [13]:
def test_args_kwargs(arg1, arg2, arg3):
        print ("인자1:", arg1)
        print ("인자2:", arg2)
        print ("인자3:", arg3)

In [15]:
# 이제 **kwargs:
kwargs = {"arg3": 3, "arg2": "two", "arg1": 5} # arg(숫자)는 위 함수의 인자의 이름과 같아야합니다.
test_args_kwargs(**kwargs)


인자1: 5
인자2: two
인자3: 3
